In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

In [1]:
#help('modules')

In [2]:
import data
import sys
import pyodbc

def sql_fn():
	try:
		conn = pyodbc.connect('Driver=ODBC Driver 17 for SQL Server;'
				#'Server=DESKTOP-SDDSOLT\SQLEXPRESS;'
				'Server=localhost;'
				'Database=hospital1;'
				'Trusted_Connection=yes;')

	except pyodbc.Error as e:
		print(f"Error connecting to MariaDB platform: {e}")
		sys.exit(1)

	cursor = conn.cursor()
	cursor.execute("DROP TABLE customers")
	cursor.execute("CREATE TABLE customers (name VARCHAR(100), value INTEGER)")
	sql = "INSERT INTO customers (name, value) VALUES (?, ?)"
	val = data.all_data("SQL")
	cursor.executemany(sql, val)
	conn.commit()
	cursor.execute('SELECT * FROM customers')
	lst_data = []
	for d in cursor:
		lst_data.append(d)

	newdata = input("Please enter the new database: ")
	if newdata == "MongoDB":
		process("SQL", lst_data)
	if newdata == "MariaDB":
		process("SQL", lst_data)
	if newdata == "ArangoDB":
		process("SQL", lst_data)

In [3]:
import data
from pymongo import MongoClient

def mongodb_fn():
    client = MongoClient(port=27017)
    db=client.business

    db.reviews.drop()
    val = data.all_data("MongoDB")
    for v in val:
        db.reviews.insert_one(v)

    cursor = db.reviews.find({})
    lst_data = []
    for item in cursor:
        noid = {i: item[i] for i in ['name','value']}
        lst_data.append(noid)

    newdata = input("Please enter the new database: ")
    if newdata == "SQL":
        process("MongoDB", lst_data)
    if newdata == "MariaDB":
        process("MongoDB", lst_data)
    if newdata == "ArangoDB":
        process("MongoDB", lst_data)

In [4]:
from pymongo import MongoClient

def process(database, data):
    client = MongoClient(port=27017)
    db=client.business
    db.reviews.drop()
    if (database == "SQL") | (database == "MariaDB"):
        for t in data:
            business = {"name": t[0], "value": t[1]}
            result = db.reviews.insert_one(business)
    if database == "ArangoDB":
        for item in data:
            result = db.reviews.insert_one(item)
    cursor = db.reviews.find({})
    result = []
    for item in cursor:
        result.append(item)
    print(result)

In [5]:
import data
import sys

def mariadb_fn():
    try:
        conn = mariadb.connect(
            user="root",
            #password="password",
            password="",
            host="127.0.0.1",
            #host="localhost",
            port=3306,
            #database="test",
            database="hospital1",
            autocommit=True
        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)

    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS hospital1.contacts")
    cur.execute("CREATE TABLE hospital1.contacts (id INT PRIMARY KEY AUTO_INCREMENT,name VARCHAR(100),value INTEGER)")

    val = data.all_data("MariaDB")
    for v in val:
        cur.execute("INSERT INTO hospital1.contacts (name, value) VALUES (?, ?)", (v[0], v[1]))
    cur.execute('SELECT * FROM hospital1.contacts')
    lst_data = []
    for c in cur:
        lst_data.append((c[1], c[2]))
    
    newdata = input("Please enter the new database: ")
    if newdata == "SQL":
        process("MariaDB", lst_data)
    if newdata == "MongoDB":
        process("MariaDB", lst_data)
    if newdata == "ArangoDB":
        process("MariaDB", lst_data)

In [7]:
import sys

def process(database, data):
    try:
        conn = mariadb.connect(
            user="root",
            #password="password",
            password="",
            #host="localhost",
            host="127.0.0.1",
            port=3306,
            #database="test",
            database="hospital1",
            autocommit=True
        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)

    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS hospital1.contacts")
    cur.execute("CREATE TABLE hospital1.contacts (id INT PRIMARY KEY AUTO_INCREMENT,name VARCHAR(100),value INTEGER)")

    if database == "SQL":
        for d in data:
            cur.execute("INSERT INTO hospital1.contacts (name, value) VALUES (?, ?)", (d[0], d[1]))
    if (database == "MongoDB") | (database == "ArangoDB"):
        for d in data:
            cur.execute("INSERT INTO hospital1.contacts(name, value) VALUES (?, ?)", (d["name"], d["value"]))

    cur.execute('SELECT * FROM hospital1.contacts')
    result_lst = []
    for c in cur:
        result_lst.append(c)
    print(result_lst)

In [6]:
#!conda install -c conda-forge python-arango
#import data
from arango import ArangoClient

def arangodb_fn():
    client = ArangoClient(hosts='http://127.0.0.1:8529/')
    #client = ArangoClient()
    #sys_db = client.db("_system", username="root", password="password")
    #sys_db = client.db("_system", username="root", password="")
    db = client.db("test", username="root", password="")
    #db = client.db("hospital1", username="root", password="")
    if db.has_collection('students'):
    #if db.has_collection('students'):
        db.delete_collection('students')
    students = db.create_collection('students')

    val = data.all_data("ArangoDB")
    students.add_hash_index(fields=["name"], unique=True)
    for v in val:
        students.insert(v)

    cursor = db.aql.execute("FOR doc IN students RETURN doc")
    lst_data = []
    for doc in cursor:
        noid = {i: doc[i] for i in ["name", "value"]}
        lst_data.append(noid)

    newdata = input("Please enter the new database: ")
    if newdata == "SQL":
        process("ArangoDB", lst_data)
    if newdata == "MongoDB":
        process("ArangoDB", lst_data)
    if newdata == "MariaDB":
        process("ArangoDB", lst_data)

In [9]:
from arango import ArangoClient

def process(database, data):
    client = ArangoClient(hosts='http://127.0.0.1:8529/')
    #sys_db = client.db("_system", username="root", password="password")
    db = client.db("test", username="root", password="")
    if db.has_collection('students'):
        db.delete_collection('students')
    students = db.create_collection('students')

    students.add_hash_index(fields=["name"], unique=True)
    if (database == "SQL") | (database == "MariaDB"):
        for d in data:
            students.insert({"name": d[0], "value": d[1]})
    if database == "MongoDB":
        for d in data:
            students.insert(d)

    cursor = db.aql.execute("FOR doc IN students RETURN doc")
    result_lst = []
    for doc in cursor:
        result_lst.append(doc)
    print(result_lst)

In [ ]:
import requests
import data
database = input("Please enter the original database: ")
if database == "SQL":
	sql_fn()
if database == "MongoDB":
	mongodb_fn()
if database == "MariaDB":
	mariadb_fn()
if database == "ArangoDB":
	arangodb_fn()